#Assignment 2 
#Gunnar Kleemann               6/26/15
#W205_1 

# set path permissions and load packages

In [ ]:
cd /Users/gunnarkleemann/Dropbox/coursework/BerkeleyDataSciences/BerkeleyCODE/205code/DataStoreRetrieval/Asn2Code

In [ ]:
import tweepy
import json
import urllib
#import sys

In [ ]:
consumer_key = XX"H1MW"
consumer_secret = XX"PDlIhV"

access_token = XX"zbUT6"
access_token_secret = XX"m3hY"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

# now acquire the tweets
* looking back into the historical tweets for a week using **Tweepy's Cursor** 
* I am writing each tweet **line by line into a CSV file** so that they can be easily stored and analyzed.  

In [ ]:
#now save output - using the manual method
import csv
import os


def TwpyCursByTime(since, until, q):
    count = 0
    countLim = 2500 # failed after 2700 tweets last time
    if not os.path.exists('TweetFldr'): 
       os.makedirs('TweetFldr')

    csvFile = open('TweetFldr/out'+since+'_'+until+'_'+q+'.csv', 'a')
    #Use csv Writer
    csvWriter = csv.writer(csvFile)

    # generate content writing line-wise
    for tweet in tweepy.Cursor(api.search,
                        q=q,
                        since=since, 
                        until=until,
                        lang="en").items():

        #Write a row to the csv file/ I use encode utf-8
        csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
        #print tweet.created_at,tweet.text # print tweet.created_at, tweet.text
        count+=1
        if count==countLim:
            return
    csvFile.close()
    next

#since="2015-06-18"
#until="2015-06-19"
#q=urllib.quote_plus("#NBAFinals2015 AND #Warriors")
#TwpyCursByTime(since,until,q)


# make a list of tuples of arguments you want to pass

In [ ]:


#since="2015-06-12"
#until="2015-06-13"

q=urllib.quote_plus("#Warriors NOT #NBAFinals2015")
q1=urllib.quote_plus("#NBAFinals2015 NOT #Warriors")
q2=urllib.quote_plus("#NBAFinals2015 AND #Warriors")




TagTimeInt=[("2015-06-12","2015-06-13",q),
            ("2015-06-12","2015-06-13",q1),
            ("2015-06-12","2015-06-13",q2),
            ("2015-06-13","2015-06-14",q),
            ("2015-06-13","2015-06-14",q1),
            ("2015-06-13","2015-06-14",q2),
            ("2015-06-14","2015-06-15",q),
            ("2015-06-14","2015-06-15",q1),
            ("2015-06-14","2015-06-15",q2),
            ("2015-06-15","2015-06-16",q),
            ("2015-06-15","2015-06-16",q1),
            ("2015-06-15","2015-06-16",q2),
            ("2015-06-16","2015-06-17",q),
            ("2015-06-16","2015-06-17",q1),
            ("2015-06-16","2015-06-17",q2),
            ("2015-06-17","2015-06-18",q),
            ("2015-06-17","2015-06-18",q1),
            ("2015-06-17","2015-06-18",q2),
            ("2015-06-18","2015-06-19",q),
            ("2015-06-18","2015-06-19",q1),
            ("2015-06-18","2015-06-19",q2)
           ]

print(TagTimeInt)

# I explicitly pause the loop to prevent from overloading the system.

In [ ]:

import time
from datetime import datetime

for n in range(len(TagTimeInt)):
    TwpyCursByTime(TagTimeInt[n][0], TagTimeInt[n][1], TagTimeInt[n][2])
    #Show the rate Limits
    print api.rate_limit_status()
    # pause before attempting next acquire
    print'2min Pause before exceding limit'
    print 'Next up '+ TagTimeInt[n][0] + TagTimeInt[n][1] + TagTimeInt[n][2]
    str(datetime.now())
    time.sleep(120)   #getting very few tweetes pause less(120sec)

# in this step I retrieve the tweets, process the text and plot the graphs

In [ ]:
import os
os.chdir('/Users/gunnarkleemann/Dropbox/coursework/BerkeleyDataSciences/BerkeleyCODE/205code/DataStoreRetrieval/Asn2Code') 
!pwd

In [ ]:
# go to the folder hoding data subfolders
os.chdir('TweetFldr')
!ls

In [ ]:
# make a list of files from each query
import os
import glob
import pandas as pd


os.chdir("/Users/gunnarkleemann/Dropbox/coursework/BerkeleyDataSciences/BerkeleyCODE/205code/DataStoreRetrieval/Asn2Code/TweetFldr/")
!pwd
SubFolder= ("tweepy_set1CSV")

FileLsNoWar=glob.glob(SubFolder+"/*NOT+%23Warriors.csv")
FileLsNoFinal=glob.glob(SubFolder+"/*NOT+%23NBAFinals2015.csv")
FileLsBOTH=glob.glob(SubFolder+"/*ND+%23Warriors.csv")


In [ ]:
# check the paths
FileLsBOTH[0:2]

In [ ]:
FileLsNoFinal[0:2]

In [ ]:
FileLsNoWar[0:2]

# Employ a function that takes in lists of files and combines them into a single dataframe

In [ ]:
def ListToDf(allFiles):
    #frame = pd.DataFrame(names=['time', 'tweettxt'], index_col=None)
    frame = pd.read_csv(allFiles[0], names=['time', 'tweettxt'], index_col=None)
    #frame.columns= ['dateTime','tweetTxt']
    for n in range(len(allFiles)):
        df= pd.read_csv(allFiles[n], names=['time', 'tweettxt'], index_col=None)
        frame=df.append(frame, ignore_index=True)
    return frame

In [ ]:
FileLsNoWar

In [ ]:
NoWar=ListToDf(FileLsNoWar)
NoFinal=ListToDf(FileLsNoFinal)
Both=ListToDf(FileLsBOTH)

In [ ]:
Both.head()

# Combine all of the tweets into one big text string ("Blob") 

In [ ]:
def BlobTheData(data):
    TweetBlob=""
    for n in range(len(data)):
         TweetBlob=TweetBlob + str(data[n])
    return TweetBlob


NoWarBlob=BlobTheData(NoWar.tweettxt)
NoFinalBlob=BlobTheData(NoFinal.tweettxt)
BothBlob=BlobTheData(Both.tweettxt)

# now tokenize, filter and count the words for each Blob

In [ ]:
NoWarBlob = "# coding=utf-8 \n" + NoWarBlob
#print NoWarBlob



In [ ]:
from nltk import word_tokenize
import nltk
import matplotlib
from nltk.tokenize import RegexpTokenizer

# tokenize everything so we dont break up words
tokenizer = RegexpTokenizer(r'\w+')
TokNoWar=tokenizer.tokenize(NoWarBlob)
TokNoFinal=tokenizer.tokenize(NoFinalBlob)
TokBoth=tokenizer.tokenize(BothBlob)

In [ ]:
# remove punctuation with string comprehension
TokNoWar = [w.lower() for w in TokNoWar if w.isalpha()]
TokNoFinal = [w.lower() for w in TokNoFinal if w.isalpha()]
TokBoth= [w.lower() for w in TokBoth if w.isalpha()]
#print words

# Filter out stop words and other contaminants

In [ ]:
stopWords = nltk.corpus.stopwords.words('english')
# add custom filter words 
OtherFilter = [u"http", u"co"]
stopWords=OtherFilter+stopWords
#print stopWords

In [ ]:
# remove stop words 
TokNoWar = [e.lower() for e in TokNoWar  if not e.lower() in stopWords]
TokNoFinal = [e.lower() for e in TokNoFinal if not e.lower() in stopWords]
TokBoth = [e.lower() for e in TokBoth if not e.lower() in stopWords]
#TokBoth
#filtered

In [ ]:
# get a distribution

DS1=nltk.FreqDist(TokNoWar)
DS1.plot(50)

In [ ]:
DS2=nltk.FreqDist(TokNoFinal)
DS2.plot(50)

In [ ]:
DS3=nltk.FreqDist(TokBoth)
DS3.plot(50)

#REDO_ Looking at my results I realize that all of them contain both tags and the "Warriors NOT NBAfinals2015" is == "NBAfinals2015 NOT warriors"!

* to address this problem I am going to use some older files where I searched on Just Warriors and just NBAfinals 2015. These are not a full week but it is all I have to run at this point 

* Use awk to remove lines that incedentally include #NBAfinals2015 in the #Warriors files and vice versa

In [ ]:
os.chdir('/Users/gunnarkleemann/Dropbox/coursework/BerkeleyDataSciences/BerkeleyCODE/205code/DataStoreRetrieval/Asn2Code/OldStuff/tweepy_firsttrycsv')
!pwd
!ls


In [ ]:
#!awk '!/pattern/' file > temp && mv temp file

!awk '!/#warriors/' "out2015-06-12_2015-06-13_%23NBAFinals2015.csv" > "out2015-06-12NBAFIN.csv" 


!awk '!/#warriors/' "out2015-06-14_2015-06-15_%23NBAFinals2015.csv" > "out2015-06-14NBAFIN.csv"

!awk '!/#warriors/' "out2015-06-13_2015-06-14_%23NBAFinals2015.csv" > "out2015-06-13NBAFIN.csv" 

!awk '!/#NBAFinals2015/' "out2015-06-12_2015-06-13_%23Warriors.csv" > "out2015-06-12Warriors.csv"     
     
!awk '!/#NBAFinals2015/' "out2015-06-13_2015-06-14_%23Warriors.csv" > "out2015-06-13Warriors.csv"     
      
!awk '!/#NBAFinals2015/' "out2015-06-14_2015-06-15_%23Warriors.csv" > "out2015-06-14Warriors.csv"     
     
!awk '!/#NBAFinals2015/' "out2015-06-15_2015-06-16_%23Warriors.csv" > "out2015-06-15Warriors.csv"     
            
# check the differences for one pre/post cleaning pair of files to be sure that we are removing lines with both tags
!diff "out2015-06-12_2015-06-13_%23NBAFinals2015.csv" "out2015-06-12NBAFIN.csv" 

# Use the cleaned files to generate a REDO of the #Warriors only and NBAfinals2015 only sets

In [ ]:
# make a list of files from each query
import os
import glob
import pandas as pd

#I moved the new files into the CleanFiles subfolder
SubFolder= ("CleanFiles")

FileLsNoWar=glob.glob(SubFolder+"/*NBAFIN.csv")
FileLsNoFinal=glob.glob(SubFolder+"/*Warriors.csv")



In [ ]:
#Employ a function that takes in lists of files and combines them into a single dataframe
NoWar=ListToDf(FileLsNoWar)
NoFinal=ListToDf(FileLsNoFinal)


In [ ]:
#Combine all of the tweets into one big text string ("Blob")
def BlobTheData(data):
    TweetBlob=""
    for n in range(len(data)):
         TweetBlob=TweetBlob + str(data[n])
    return TweetBlob


NoWarBlob=BlobTheData(NoWar.tweettxt)
NoFinalBlob=BlobTheData(NoFinal.tweettxt)

In [ ]:
#now tokenize, filter and count the words for each Blob

# tokenize everything so we dont break up words
tokenizer = RegexpTokenizer(r'\w+')
TokNoWar=tokenizer.tokenize(NoWarBlob)
TokNoFinal=tokenizer.tokenize(NoFinalBlob)

# remove punctuation with string comprehension
TokNoWar = [w.lower() for w in TokNoWar if w.isalpha()]
TokNoFinal = [w.lower() for w in TokNoFinal if w.isalpha()]

#Filter out stop words and other contaminants
stopWords = nltk.corpus.stopwords.words('english')
# add custom filter words 
OtherFilter = [u"http", u"co"]
stopWords=OtherFilter+stopWords

# remove stop words 
TokNoWar = [e.lower() for e in TokNoWar  if not e.lower() in stopWords]
TokNoFinal = [e.lower() for e in TokNoFinal if not e.lower() in stopWords]
TokBoth = [e.lower() for e in TokBoth if not e.lower() in stopWords]



In [ ]:
# get a distribution

DS1=nltk.FreqDist(TokNoWar)
DS1.plot(50)

In [ ]:
DS2=nltk.FreqDist(TokNoFinal)
DS2.plot(50)